In [123]:
import cv2
import numpy as np
import skimage as ski # <- na potrzeby generowania kształtów itp.
import typing as _t

## Zadanie
Proszę przygotować program, który będzie śledził poruszanie się bil na stole 
do snookera. Plikiem źródłowym będzie snooker2.mp4. 

Kroki przetwarzania:

### 1. Odfiltrowanie tła
Poprzez progowanie zakresu koloru znajdź bile na stole. Kolorem bazowym będzie
kolor stołu. Punkty, których kolor różni się więcej niż tol od średniego koloru
stołu ustaw jako punkty obiektów, pozostałe - jako punkty tła. Wykonaj podobnie
jak w Lab3 - Samolot.

In [18]:
class background_filter:

	def __init__(self, bg_cutout: np.ndarray, *, boost_dist=(0, 0, 0)) -> None:
		"""
		Inicjalizacja klasy do usuwania tła - przyjmuje wycinek tła, z którego
		buduje średni kolor tła oraz maksymalną różnicę średniego tła względem
		wycinka - te używane są później do usuwania tła z klatek nagrania.

		Opcjonalny argument [boost_dist] pozwala przesunąć maksymalną różnicę,
		co przydaje się ze względu na ciemniejsze tło w narożnikach stołu
		do bilarda
		"""

		self.color_bg: np.ndarray = np.uint8(
			np.floor(np.mean(bg_cutout, axis=(0, 1)))
		)
		diffs: np.ndarray = np.abs(np.int16(bg_cutout) - self.color_bg)
		self.max_dist: np.ndarray = diffs.max(axis=(0, 1)) \
					+ np.array(boost_dist)

	def background_removal_mask(self, frame: np.ndarray, *, kernel=np.array([])) -> np.ndarray:
		"""
		Używając parametrów tej klasy, tworzy maskę do usunięcia tła z obrazu;
		na masce kolor biały (wartość 255) oznacza tło, czarny resztę obrazu.

		Opcjonalny argument [kernel] pozwala dodatkowo przeprocesować maskę
		przez filtr [cv2.dilate]
		"""

		masks = np.abs(np.int16(frame) - self.color_bg) < self.max_dist
		mask_bin = np.logical_and(
			np.logical_and(
				masks[:, :, 0],
				masks[:, :, 1],
			),
			masks[:, :, 2]
		)

		mask = np.uint8(mask_bin * 255)
		if kernel.shape != (0, ):
			mask = cv2.dilate(mask, kernel, iterations=1)

		return mask


# Maska przysłaniająca - zakrywa koszyki w stole oraz tło poza stołem,
# suplementując maskę usuwającą kolor tła
out_mask = np.zeros((720, 1280), dtype=np.uint8)
out_mask[140:-140, 210:-210] = 255
out_mask[:165, :235] = 0
out_mask[:165, -235:] = 0
out_mask[-165:, :235] = 0
out_mask[-165:, -235:] = 0
out_mask[:165, 625:-625] = 0
out_mask[-165:, 625:-625] = 0

### 2. Detekcja obiektów
Do znalezienia bil proszę użyć obiektu 
[SimpleBlobDetector](https://learnopencv.com/blob-detection-using-opencv-python-c/)

Proszę ustawić odpowiednie parametry, tak aby znajdowane były obiekty
o określonym przedziale wielkości i współczynniku okrągłości. Na poniższym
rysunku widzimy obrysy znalezionych bil.

In [19]:
def detect_balls() -> cv2.SimpleBlobDetector:
	params = cv2.SimpleBlobDetector.Params()

	params.minThreshold = 0
	params.maxThreshold = 200

	params.filterByArea = True
	params.maxArea = 600

	params.filterByCircularity = True
	params.minCircularity = 0.7

	params.filterByConvexity = True
	params.minConvexity = 0.8

	params.filterByInertia = True
	params.minInertiaRatio = 0.2

	return cv2.SimpleBlobDetector.create(params)

### 3. Śledzenie kul
Ostatnim krokiem jest śledzenie kul. Źródłem danych jest lista obiektów 
znalezionych w danej ramce. Należy zbudować słownik, w którym będziemy
przechowywać dane w postaci:
```python
Kule = {
	1: ([(x,y), (x,y), (x,y), (x,y), (x,y), (x,y), (x,y), (x,y)], False),
	2: ([(x,y), (x,y), (x,y), (x,y), (x,y), (x,y)], True),
	3: ([(x,y), (x,y), (x,y), (x,y), (x,y), (x,y), (x,y), (x,y), (x,y)], True),
}
```
gdzie kluczem jest kolejny znaleziony obiekt, a wartością lista pozycji tego
obiektu na całym filmie i znacznik aktualizacji. Klucze są inkrementowane.

Przypadki:
- kula się pojawia (na poprzedniej ramce w tej lokalizacji (+/- 10 pikseli) 
  nie było żadnego elementu - tworzymy nowy wpis w słowniku, znacznik = True
- kula stoi (w słowniku znajduje się gdzieś ta sama pozycja (x,y) - wystarczy 
  sprawdzać ostatni element listy - słownika nie aktualizujemy, znacznik = True
- kula poruszyła się (w słowniku na końcu którejś listy jest w pobliżu 
  (+/-10 pikseli) element - do odpowiedniej listy dopisujemy nową pozycję, 
  znacznik = True
- kula zniknęła. Element słownika, który nie został zaktualizowany, otrzymuje
  znacznik False, a w następnym obiegu przenoszony jest do słownika archiwalnego.



In [ ]:
aa = 1

## Główna pętla programu:

In [155]:
erode_operator = _t.TypedDict("erode_operator", { "erode": np.ndarray })
dilate_operator = _t.TypedDict("dilate_operator", { "dilate": np.ndarray })
	
def erode_dilate_pipeline(src: np.ndarray, ops: _t.List[_t.Union[erode_operator, dilate_operator]]) -> np.ndarray:
	result = src
	for op in ops:
		if "erode" in op:
			result = cv2.erode(result, op["erode"], iterations= 1)
		if "dilate" in op:
			result = cv2.dilate(result, op["dilate"], iterations= 1)

	return result



def dilate_submask(submask: np.ndarray, kernel: np.ndarray) -> np.ndarray:
	result = np.uint8(submask * 255)
	result = cv2.dilate(result, kernel, iterations= 1)

	return result

def apply_submask(submask: np.ndarray, mask: np.ndarray) -> np.ndarray:
	result = np.bitwise_and(submask, mask)

	return result

def ball_keyframes(detector: cv2.SimpleBlobDetector, where: np.ndarray, canvas: np.ndarray, *, color= (0, 255, 0)):
	key_pts = detector.detect(where)

	img_with_key_pts: np.ndarray = cv2.drawKeypoints(
		canvas, key_pts, np.array([]), color,
		flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
	)

	return key_pts, img_with_key_pts

cap = cv2.VideoCapture("lab5/Snooker2.mp4")

keep = True
init = True
# Operatory do erosion i dilation
kr_disk_3 = ski.morphology.disk(3)
kr_disk_2 = ski.morphology.disk(2)
kr_disk_1 = ski.morphology.disk(1)
kr_pixel_2 = np.ones((2, 2))
kr_pixel_1 = np.array([1])


bg_filter: background_filter
detector = detect_balls()

while keep:
	# t = time.time()
	flag, frame = cap.read()
	if flag:
		# Inicjalizacja - pobranie wycinka stołu z pierwszej klatki i utworzenie z jego pomocą
		# obiektu do usuwania tła
		if init:
			h, w, _ = frame.shape
			off_h = int((h - 320) / 2)
			off_w = int((w - 320) / 2)
			fr_cut = frame[
				off_h : -off_h,
				off_w : -off_w
			]
			bg_filter = background_filter(fr_cut, boost_dist=(5, 15, 5))
			init = False

		pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

		# Utworzenie maski tła, odwrócenie jej na maskę elementów i odcięcie elementów poza stołem
		mask = 255 - bg_filter.background_removal_mask(frame, kernel= kr_disk_2)
		mask = np.bitwise_and(mask, out_mask)

		# Klatka w skali HSL - pozwoli podzielić bile w zależności od koloru
		frame_det = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)

		frame_big_saturation = frame_det[:, :, 1] > 190  # Biała bila
		frame_low_value = frame_det[:, :, 1] < 40  # Czarna bila

		frame_big_saturation = erode_dilate_pipeline(
			np.uint8(frame_big_saturation * 255),
			[
				{ "erode": kr_disk_1}
			]
		)

		frame_low_value = erode_dilate_pipeline(
			np.uint8(frame_low_value * 255),
			[
				{ "dilate": kr_pixel_2 },
				{ "erode": kr_pixel_1 },
				{ "dilate": kr_disk_1 },
			]
		)

		frame_rest = np.logical_not(
			np.logical_or(frame_big_saturation, frame_low_value)
		)  # kolorowe bile

		frame_rest = erode_dilate_pipeline(
			np.uint8(frame_rest * 255),
			[
				{ "erode": kr_disk_3 },
				{ "dilate": kr_disk_2 },
				{ "erode": kr_disk_2 },
				{ "dilate": kr_disk_2 },
			]
		)


		frame_big_saturation = apply_submask(frame_big_saturation, mask)
		frame_low_value = apply_submask(frame_low_value, mask)
		frame_rest = apply_submask(frame_rest, mask)

		frame_hsv = frame_det

		frame_det = cv2.cvtColor(frame_det, cv2.COLOR_HSV2BGR)

		# kpt_white, frame_big_saturation = ball_keyframes(
		# 	detector,
		# 	frame_big_saturation, 
		# 	frame_big_saturation
		# )
		# kpt_dark, frame_low_value = ball_keyframes(
		# 	detector, 
		# 	frame_low_value, 
		# 	frame_low_value
		# )
		# kpt_color, frame_rest = ball_keyframes(
		# 	detector, 
		# 	frame_rest,
		# 	frame_rest
		# )

		frame[np.bool_(255 - out_mask), 0] = 0
		frame[np.bool_(255 - out_mask), 1] = 0
		frame[np.bool_(255 - out_mask), 2] = 0
		cv2.imshow("Frame", frame)
		cv2.imshow("Frame white", frame_big_saturation)
		cv2.imshow("Frame dark", frame_low_value)
		cv2.imshow("Frame colors", frame_rest)

	else:
		cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame - 1)
		print("Video finished")
		keep = False
		cv2.destroyAllWindows()
		cap.release()

	if cv2.waitKey(1) == 27:
		keep = False
		cv2.destroyAllWindows()
		cap.release()
		break

Video finished
